In [ ]:
%pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import pywt
import numpy as np

# Load and normalize MNIST dataset #x_train (60000, 28, 28) x_test (10000, 28, 28) y_train (60000,) y_test (10000,)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("x train loaded and normalized: " + str(np.shape(x_train)))
print("x test loaded and normalized: " + str(np.shape(x_test)))
print("y train loaded and normalized: " + str(np.shape(y_train)))
print("y test loaded and normalized: " + str(np.shape(y_test)))
print("")

# Apply wavelet transform to the data  #x_train (60000, 14, 14) x_test (10000, 14, 14)
x_train_wavelet = [pywt.dwt2(x, 'db1')[0] for x in x_train]
x_test_wavelet = [pywt.dwt2(x, 'db1')[0] for x in x_test]
print("x train wavelet: " + str(np.shape(x_train_wavelet)))
print("x test wavelet: " + str(np.shape(x_test_wavelet)))
print("")

# Reshape data for input into CNN #x_train (60000, 14, 14, 1) x_test (10000, 14, 14, 1)
x_train_wavelet = tf.reshape(x_train_wavelet, (len(x_train_wavelet), 14, 14, 1))
x_test_wavelet = tf.reshape(x_test_wavelet, (len(x_test_wavelet), 14, 14, 1))
print("x train wavelet reshaped: " + str(np.shape(x_train_wavelet)))
print("x test wavelet reshaped: " + str(np.shape(x_test_wavelet)))
print("")

# One-hot encode labels #y_train (60000, 10) y_test (10000, 10)
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)
print("y train one hot coded: " + str(np.shape(y_train_one_hot)))
print("y test one hot coded: " + str(np.shape(y_test_one_hot)))

# Define CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(14, 14, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and fit model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_wavelet, y_train, epochs=5)

# Evaluate model on test set
test_loss, test_acc = model.evaluate(x_test_wavelet, y_test)
print('Test accuracy:', test_acc)

#The wavelet transform function pywt.dwt2() returns a tuple containing two elements, the wavelet coefficients and the detail coefficients. 
#The wavelet coefficients are represented as a 2D array, which has a smaller size than the original input array.

#For example, if x is a 2D array with shape (28, 28), pywt.dwt2(x, 'db1') will return a tuple (cA, cD), where cA is the wavelet coefficients and cD is the detail coefficients. 
#The wavelet coefficients cA will have shape (14, 14), which is smaller than the original input array x.

#Therefore, when you apply the wavelet transform to the MNIST data using x_train_wavelet = [pywt.dwt2(x, 'db1')[0] for x in x_train], the resulting wavelet-transformed data will have a smaller size than the original data.
#If you want to preserve the original size of the input data, you can use the inverse wavelet transform function pywt.idwt2() to reconstruct the original data from the wavelet coefficients.
#For example, you can use the following code to reconstruct the original data from the wavelet coefficients:
#x_train_reconstructed = [pywt.idwt2(x, np.zeros_like(x), 'db1') for x in x_train_wavelet]
#x_test_reconstructed = [pywt.idwt2(x, np.zeros_like(x), 'db1') for x in x_test_wavelet]




x train loaded and normalized: (60000, 28, 28)
x test loaded and normalized: (10000, 28, 28)
y train loaded and normalized: (60000,)
y test loaded and normalized: (10000,)

x train wavelet: (60000, 14, 14)
x test wavelet: (10000, 14, 14)

x train wavelet reshaped: (60000, 14, 14, 1)
x test wavelet reshaped: (10000, 14, 14, 1)

y train one hot coded: (60000, 10)
y test one hot coded: (10000, 10)
Epoch 1/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2024 - accuracy: 0.9422
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0750 - accuracy: 0.9772
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0553 - accuracy: 0.9829
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0440 - accuracy: 0.9862
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0432 - accuracy: 0.9866
Test accuracy: 0.9865999817848206
